In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Input data
data = [
    ["https://www.tui.pl/wypoczynek/wyspy-kanaryjskie/teneryfa/gf-isabel-tfs31021/...", 11168, "2 posiłki", "warszawa", "teneryfa", 26, 20, "nie", "tak", "nie", "jakieś", 5.50, 20, 8],
    ["https://www.tui.pl/wypoczynek/wyspy-kanaryjskie/teneryfa/gf-isabel-tfs31021/...", 13716, "all inclusive", "warszawa", "teneryfa", 26, 20, "tak", "tak", "nie", "jakieś", 5.50, 20, 8],
    ["https://www.tui.pl/wypoczynek/wyspy-kanaryjskie/gran-canaria/gloria-palace-san-agustin-thalasso-hotel-...", 11542, "2 posiłki", "warszawa", "gran canaria", 26, 20, "nie", "tak", "nie", "dużo", 5.40, 27, 6],
    ["https://www.tui.pl/wypoczynek/wyspy-kanaryjskie/gran-canaria/gloria-palace-san-agustin-thalasso-hotel-...", 13838, "all inclusive", "warszawa", "gran canaria", 26, 20, "tak", "tak", "nie", "dużo", 5.40, 27, 6],
    ["https://www.tui.pl/wypoczynek/hiszpania/majorka/tui-suneo-santa-ponsa-pmi31035/...", 8712, "", "warszawa", "majorka", 22, 18, "tak", "tak", "nie", "tylko basen", 3.25, 30, 5],
    ["https://www.wakacje.pl/oferty/grecja/kreta/panormos/iberostar-creta-panorama-mare-...", 10433, "", "wrocław", "kreta", 20, 16, "tak", "nie", "", "tylko basen", np.nan, 7, 7],
    ["https://www.itaka.pl/wczasy/wyspy-kanaryjskie/lanzarote/hotel-thb-tropical-island,...", 13876, "all inclusive", "warszawa", "Lanzarote", np.nan, np.nan, "tak", "tak", "tak", "jakieś", 5.50, 30, 8]
]

columns = [
    "link", "cena", "komentarz", "skąd wylot", "gdzie?", "temperatura powietrza", "temperatura morza",
    "all inclusive?", "bagaż w cenie?", "transfer w cenie?", "zjeżdżalnie w hotelu i okolicy",
    "czas lotu", "odległość od lotniska w km", "urokliwość hotelu"
]

df = pd.DataFrame(data, columns=columns)

# Scoring scales
max_cena = df["cena"].max()
df["punkty_cena"] = (1 - (df["cena"] / max_cena)) * 10

df["punkty_all_inclusive"] = df["all inclusive?"].apply(lambda x: 10 if x == "tak" else 0)
df["punkty_bagaż"] = df["bagaż w cenie?"].apply(lambda x: 10 if x == "tak" else 0)
df["punkty_transfer"] = df["transfer w cenie?"].apply(lambda x: 10 if x == "tak" else 0)

zjezdzalnie_scores = {"dużo": 10, "tylko basen": 5, "jakieś": 3, "": 0, None: 0}
df["punkty_zjeżdżalnie"] = df["zjeżdżalnie w hotelu i okolicy"].map(zjezdzalnie_scores).fillna(0)

df["punkty_odległość"] = (1 - (df["odległość od lotniska w km"] / df["odległość od lotniska w km"].max())) * 10
df["punkty_czas_lotu"] = (1 - (df["czas lotu"] / df["czas lotu"].max())) * 10
df["punkty_urokliwość"] = df["urokliwość hotelu"] * 1

# Comfortable air temperature (25-27 is ideal)
df["punkty_temp_powietrze"] = df["temperatura powietrza"].apply(
    lambda x: 10 if 25 <= x <= 27 else max(0, 10 - abs(26 - x)) if pd.notnull(x) else 0
)

# Comfortable sea temperature (20-22 is ideal)
df["punkty_temp_morze"] = df["temperatura morza"].apply(
    lambda x: 10 if 20 <= x <= 22 else max(0, 10 - abs(21 - x)) if pd.notnull(x) else 0
)

# Sum of points
df["suma_punktów"] = df[
    [
        "punkty_cena", "punkty_all_inclusive", "punkty_bagaż", "punkty_transfer",
        "punkty_zjeżdżalnie", "punkty_odległość", "punkty_czas_lotu",
        "punkty_urokliwość", "punkty_temp_powietrze", "punkty_temp_morze"
    ]
].sum(axis=1)

# Sorting offers by score
df_sorted = df.sort_values(by="suma_punktów", ascending=False)

# Display sorted DataFrame
display(df_sorted)



,link,cena,komentarz,skąd wylot,gdzie?,temperatura powietrza,temperatura morza,all inclusive?,bagaż w cenie?,transfer w cenie?,...,punkty_all_inclusive,punkty_bagaż,punkty_transfer,punkty_zjeżdżalnie,punkty_odległość,punkty_czas_lotu,punkty_urokliwość,punkty_temp_powietrze,punkty_temp_morze,suma_punktów
3,https://www.tui.pl/wypoczynek/wyspy-kanaryjski...,13838,all inclusive,warszawa,gran canaria,26.0,20.0,tak,tak,nie,...,10,10,0,10,1.000000,0.181818,6,10.0,10.0,57.209204
1,https://www.tui.pl/wypoczynek/wyspy-kanaryjski...,13716,all inclusive,warszawa,teneryfa,26.0,20.0,tak,tak,nie,...,10,10,0,3,3.333333,0.000000,8,10.0,10.0,54.448640
4,https://www.tui.pl/wypoczynek/hiszpania/majork...,8712,,warszawa,majorka,22.0,18.0,tak,tak,nie,...,10,10,0,5,0.000000,4.090909,5,6.0,7.0,50.812443
2,https://www.tui.pl/wypoczynek/wyspy-kanaryjski...,11542,2 posiłki,warszawa,gran canaria,26.0,20.0,nie,tak,nie,...,0,10,0,10,1.000000,0.181818,6,10.0,10.0,48.863859
0,https://www.tui.pl/wypoczynek/wyspy-kanaryjski...,11168,2 posiłki,warszawa,teneryfa,26.0,20.0,nie,tak,nie,...,0,10,0,3,3.333333,0.000000,8,10.0,10.0,46.284904
5,https://www.wakacje.pl/oferty/grecja/kreta/pan...,10433,,wrocław,kreta,20.0,16.0,tak,nie,,...,10,0,0,5,7.666667,NaN,7,4.0,5.0,41.147929
6,https://www.itaka.pl/wczasy/wyspy-kanaryjskie/...,13876,all inclusive,warszawa,Lanzarote,NaN,NaN,tak,tak,tak,...,10,10,10,3,0.000000,0.000000,8,0.0,0.0,41.000000
